## Smart Contracts and Tokens (Algos)
#### 07.3 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-09

* Identify public address of a smart contract
* Fund stateful smart contracts
* Access token holdings ...
    * of smart contract itself
    * of individual users

## Setup
See notebook 04.1, loading `algo_util.py`, the five accounts and the Purestake credentials
* Consider hiding the code below

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json
import base64
import datetime

In [3]:
from pyteal import *

In [13]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19954417


In [5]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


## Our project: Richkid
* Idea: access the holdings of a user
* Compare to the holdings of the smart contact
    * A *Richkid* is user with more ALGOs than the smart contract
    * A *Poorkid* is a user with fewer ALGOs than the smart contract

#### Step 1: Define Approval program

In [6]:
handle_creation = Seq(
    [
        # Initialize GLOBAL variables
        App.globalPut(Bytes("FundingAddr"), Global.current_application_address()),            
        App.globalPut(Bytes("Funds"),       Balance(Global.current_application_address())),            
        App.globalPut(Bytes("Url"),         Bytes("www.usi.ch")),
        App.globalPut(Bytes("Note"),        Bytes("Hello world!")),
        Return(Int(1)),                                          # Return "OK"
    ]
)

# Optin
handle_optin = Seq (
    [
        App.localPut(Int(0), Bytes("YourHolding"), Balance(Txn.sender())),
        If( Balance(Txn.sender()) > Balance(Global.current_application_address()),
            App.localPut(Int(0), Bytes("YourStatus"), Bytes("Richkid")),
            App.localPut(Int(0), Bytes("YourStatus"), Bytes("Poorkid"))
           ),
        Return(Int(1))
    ]
)


handle_closeout = Seq(
    [
        Return(Int(1)),
    ]
)

handle_updateapp = Return( 
    Txn.sender() == Global.creator_address()    # only TRUE if delete request is made by creator
)

handle_deleteapp = Return(
    Txn.sender() == Global.creator_address()    # only TRUE if delete request is made by creator
)

# handle interaction
handle_noop = Seq(
    [
        App.localPut(Int(0), Bytes("YourHolding"), Balance(Txn.sender())),
        If( Balance(Txn.sender()) > Balance(Global.current_application_address()),
            App.localPut(Int(0), Bytes("YourStatus"), Bytes("Richkid")),
            App.localPut(Int(0), Bytes("YourStatus"), Bytes("Poorkid"))
           ),
        Return(Int(1))
    ]
)

In [7]:
richkid_approval_pyteal = Cond(
    [Txn.application_id() == Int(0),              handle_creation],
    [Txn.on_completion()  == OnComplete.OptIn,    handle_optin],
    [Txn.on_completion()  == OnComplete.CloseOut, handle_closeout],
    [Txn.on_completion()  == OnComplete.UpdateApplication, handle_updateapp],
    [Txn.on_completion()  == OnComplete.DeleteApplication, handle_deleteapp],
    [Txn.on_completion()  == OnComplete.NoOp, handle_noop],
)

#### Compile PyTEAL -> TEAL
* Notice the `Mode.Application` (was `Mode.Signature`)

In [8]:
richkid_approval_teal = compileTeal(richkid_approval_pyteal,mode=Mode.Application, version=5)
#print(richkid_approval_teal)

#### Step 1b: Define Clear State program
* This program handles forced opt-outs

In [9]:
richkid_clear_pyteal = Seq(
    [
        Return(Int(1)),
    ]
)

In [10]:
richkid_clear_teal = compileTeal(richkid_clear_pyteal,mode=Mode.Application, version=3)
# print(richkid_clear_teal)

#### Compile TEAL -> Bytecode
This is slightly different ... we need one additional step for Byte-encoding

In [14]:
richkid_approval_b64 = algod_client.compile(richkid_approval_teal)
Richkid_Approval =  base64.b64decode(richkid_approval_b64['result'])

richkid_clear_b64 = algod_client.compile(richkid_clear_teal)
Richkid_Clear =  base64.b64decode(richkid_clear_b64['result'])

## Deploy Smart Contract

##### Bob deploys the smart contract
* Reserve (global/local) storage with `StateSchema`
* New command `ApplicationCreateTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCreateTxn)

In [15]:
# Step 1: Prepare the transaction
sp = algod_client.suggested_params()

# How much space do we need?
global_ints = 1   # for "Funds"
global_bytes = 3   # for "FundignAddr", "Url", "Note"
richkid_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 1     # for "YourHolding"
local_bytes = 1    # for "YourStatus"
richkid_local_schema = transaction.StateSchema(local_ints, local_bytes)

txn = transaction.ApplicationCreateTxn(
      sender = Bob['public'],                # <-- sender public
      sp = sp,                               # <-- sp
      on_complete = 0,                       # <- what to do when finished (nothing)
      approval_program = Richkid_Approval,   # <-- approval program 
      clear_program = Richkid_Clear,         # <-- clear program 
      global_schema = richkid_global_schema, # <-- reserve global space 
      local_schema = richkid_local_schema    # <-- reserve local space
    )

In [ ]:
# Step 2: sign transaction
stxn = txn.sign(Bob['private'])

# Step 3: send
txid=algod_client.send_transactions([stxn])

# Step 4: wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

In [ ]:
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)

## The public address of the smart contract
* Can be found in the global state

In [21]:
format_state(read_global_state(algod_client,app_id))

{'FundingAddr': 'GN3H5KMUXFLGDT6JORN5FEBMHVKVNLM4LJWWZKQNHFSCJOGJHOP5DOPWWU',
 'Funds': 0,
 'Note': 'Hello world!',
 'Url': 'www.usi.ch'}

In [23]:
app_addr = format_state(read_global_state(algod_client,app_id))['FundingAddr']
print(app_addr)

GN3H5KMUXFLGDT6JORN5FEBMHVKVNLM4LJWWZKQNHFSCJOGJHOP5DOPWWU


## Fund the smart contract
* Funding works like funding any other address, either via bank or via a `PaymentTxn()`
* https://bank.testnet.algorand.network

#### After funding, the Funds of the Smart contract will still be shown as `0`
* The smart contract updates its global state only during
    * Opt-in
    * Noop

In [25]:
format_state(read_global_state(algod_client,app_id))

{'FundingAddr': 'GN3H5KMUXFLGDT6JORN5FEBMHVKVNLM4LJWWZKQNHFSCJOGJHOP5DOPWWU',
 'Note': 'Hello world!',
 'Url': 'www.usi.ch',
 'Funds': 0}

## Opt-In

In [ ]:
user = Alice

# Step 1: prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationOptInTxn(user['public'], sp, app_id)

# Step 2: sign transaction
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: await confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Watch the state of the contract evolve

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,user['public'],app_id)

#### Now try
* For Bob to opt-in
* For Charlie to opt in

## Users can call the Smart contract ("make a visit")
* Strangley, calling a Smart Contact is done with a `ApplicationNoOpTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationNoOpTxn)

In [193]:
user = Charlie

# Step 1: prepare
sp = algod_client.suggested_params()
txn = transaction.ApplicationNoOpTxn(user['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: wait for condfirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19934165.
Waiting for round 19934165 to finish.
Waiting for round 19934166 to finish.
Transaction 4ZFDUJMORKQD46JJR27WRG5RHZPJQDA3YLXNFDJ4XJN7IW4E7PVQ confirmed in round 19934167.


#### Watch the state of the contract evolve

In [194]:
format_state(read_global_state(algod_client,app_id))

{'Funding': '3L2VQ4MANUNV2OR7Z2NEN3DMUE63ALGPNXXBUCPCWI6AMEO4XICKORQC64',
 'Funds': 8998000,
 'Info': 'www.usi.ch',
 'All_visits': 2,
 'Fans': 1,
 'Note': 'Hello world!'}

In [195]:
read_local_state(algod_client,user['public'],app_id)

{'Visits': 2,
 'LastHolding': 15155000,
 'YourHolding': 15654000,
 'Message': 'You are welcome'}

## Users close out (leave) App
* With a `ApplicationCloseOutTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCloseOutTxn)

In [ ]:
user = Alice

# Step 1: prepare
sp = algod_client.suggested_params()
txn = transaction.ApplicationCloseOutTxn(user['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: wait for condfirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Watch the state of the contract evolve

In [ ]:
format_state(read_global_state(algod_client,app_id))

In [ ]:
read_local_state(algod_client,user['public'],app_id)

## Deleting the app
* Rather important, because an address can only create **10 apps**
* App can be deleted by creator

In [196]:
creator = Bob

# Step 1: Prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationDeleteTxn(creator['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(creator['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step4 : wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# display results
print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])

Current round is  19949122.
Waiting for round 19949122 to finish.
Waiting for round 19949123 to finish.
Transaction BTHZWA7NTDYY6D7QRF3SEHN5LQZPU2ZE6BBCXD2V5JB5U4OA5HHA confirmed in round 19949124.
Deleted app-id: 73707385


## Appendix: the public address of a Smart Contract
* Is there a chance to get the address without writing it to the global state?
* In theory, there should exist the following function, but it does not: <br>
`algosdk.logic.get_application_address(app_id)    # <--- not (yet) available`

#### Here is a workaround:

In [18]:
# The address can be calculated using the following code from the smart contract’s application id.
# from https://developer.algorand.org/docs/get-details/dapps/smart-contracts/apps/
import algosdk.encoding
app_addr = algosdk.encoding.encode_address(algosdk.encoding.checksum(b'appID'+(app_id).to_bytes(8, 'big')))
print( app_addr )

GN3H5KMUXFLGDT6JORN5FEBMHVKVNLM4LJWWZKQNHFSCJOGJHOP5DOPWWU
